In [10]:
import xml.etree.ElementTree as ET
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

data = os.listdir('/root/autodl-tmp/data/monuseg/MoNuSegTestData')
data.sort()
data = data[:-3]
print(data)

monuseg_labels  = [l for l in data if l.endswith('.xml')]
print(len(monuseg_labels))
imgs_names = [l for l in data if l.endswith('.tif')]
print(len(imgs_names))
labels = np.zeros((0,1000, 1000, 2), dtype=np.int32)

for name in monuseg_labels:
    tree = ET.parse('./' + name)
    root = tree.getroot()
    regions = root.find('Annotation').find('Regions')
    mask = np.zeros((1000, 1000), dtype=np.int32)
    instance_mask = np.zeros((1000, 1000), dtype=np.int32)

    idx =1
    for region in regions.findall('Region'):
        arr = np.array([],dtype=np.int32)
        vertices = region.find('Vertices')
        for vertice in vertices.findall('Vertex'):
            x = round(float(vertice.get('X')))
            y = round(float(vertice.get('Y')))
            arr = np.append(arr, [x, y])
            arr = arr.reshape(-1, 2)
        
        ins = np.zeros((1000, 1000), dtype=np.int32)
        
        cv2.fillPoly(mask, [arr], 1)
        cv2.fillPoly(ins, [arr], 1)
        ins = ins * idx
        ins[np.logical_and(instance_mask!=0,ins!=0)] = 0
        instance_mask += ins
        idx += 1
    label = np.stack([ instance_mask, mask], axis=-1)
    labels = np.concatenate([labels, label[np.newaxis, ...]], axis=0)
np.save('/root/autodl-tmp/data/monuseg/MoNuSegTestData/labels.npy',labels)

['TCGA-2Z-A9J9-01A-01-TS1.tif', 'TCGA-2Z-A9J9-01A-01-TS1.xml', 'TCGA-44-2665-01B-06-BS6.tif', 'TCGA-44-2665-01B-06-BS6.xml', 'TCGA-69-7764-01A-01-TS1.tif', 'TCGA-69-7764-01A-01-TS1.xml', 'TCGA-A6-6782-01A-01-BS1.tif', 'TCGA-A6-6782-01A-01-BS1.xml', 'TCGA-AC-A2FO-01A-01-TS1.tif', 'TCGA-AC-A2FO-01A-01-TS1.xml', 'TCGA-AO-A0J2-01A-01-BSA.tif', 'TCGA-AO-A0J2-01A-01-BSA.xml', 'TCGA-CU-A0YN-01A-02-BSB.tif', 'TCGA-CU-A0YN-01A-02-BSB.xml', 'TCGA-EJ-A46H-01A-03-TSC.tif', 'TCGA-EJ-A46H-01A-03-TSC.xml', 'TCGA-FG-A4MU-01B-01-TS1.tif', 'TCGA-FG-A4MU-01B-01-TS1.xml', 'TCGA-GL-6846-01A-01-BS1.tif', 'TCGA-GL-6846-01A-01-BS1.xml', 'TCGA-HC-7209-01A-01-TS1.tif', 'TCGA-HC-7209-01A-01-TS1.xml', 'TCGA-HT-8564-01Z-00-DX1.tif', 'TCGA-HT-8564-01Z-00-DX1.xml', 'TCGA-IZ-8196-01A-01-BS1.tif', 'TCGA-IZ-8196-01A-01-BS1.xml', 'TCGA-ZF-A9R5-01A-01-TS1.tif', 'TCGA-ZF-A9R5-01A-01-TS1.xml']
14
14


In [11]:
import torch
import os
import numpy as np
import PIL.Image as Image

imgs = torch.zeros((0, 3, 1000, 1000), dtype=torch.float32)

for name in imgs_names:
    img = Image.open('./' + name)
    img = np.array(img)
    img = torch.from_numpy(img).float()/255
    img = img.permute(2, 0, 1)
    img = img.unsqueeze(0)
    imgs = torch.cat([imgs, img], dim=0)
torch.save(imgs,'/root/autodl-tmp/data/monuseg/MoNuSegTestData/imgs.pt')
